In [ ]:
!pip install fake_useragent

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent
from tqdm.auto import tqdm
import pandas as pd
import numpy as np

In [ ]:
ua = UserAgent()
session = requests.session()

In [ ]:
def parse_one_article(block):

    # создаем запрос для отдельной страницы с рецептом

    req = session.get(block['ссылка на статью'],
                      headers={'User-Agent': ua.random})
    page = req.text

    # парсим страницу
    try:

        soup = BeautifulSoup(page, 'html.parser')
        text = soup.get_text()

    # проверяем наличие слова "калорийность" в тексте
        if 'ккал' in text:

            block['калорийность (ккал)'] = int(soup.find('span',
                                                        {'itemprop':
                                                        'calories'}).text)
            block['белки (г)'] = int(soup.find('span',
                                            {'id': 'nutr_p'}).text)
            block['жиры (г)'] = int(soup.find('span',
                                            {'id':'nutr_f'}).text)
            block['углеводы (г)'] = int(soup.find('span',
                                                {'id':
                                                'nutr_c'}).text)
            ingredients = soup.find_all('div', class_='ingredient list-item')

            block['ингредиенты'] = []
            for ingredient_element in ingredients:
                ingredient = ingredient_element.find(
                    'meta', itemprop='recipeIngredient')['content']
                ingredient = ingredient.split(' - ')[0]
                block['ингредиенты'].append(ingredient)

    # поскольку текст рецепта разбит по блокам, то перед добавлением
    # в датафрейм необходимо собрать его части в одну строку, при этом почистив
    # части от личшних пробелов и переносов строк.

            block['рецепт'] = []
            recipe = soup.find_all('p', {'class': 'instruction'})
            for step in recipe:
                block['рецепт'].append(step.text)
            block['рецепт'] = ' '.join(block['рецепт'])

    except Exception:
        pass

    return block


In [ ]:
def get_nth_page(page_number):

    # создаем запрос для страницы с рецептами

    url = f'https://1000.menu/catalog/bjstro-i-vkusno/{page_number}'
    req = session.get(url, headers={'User-Agent': ua.random})
    page = req.text

    # парсим страницу

    soup = BeautifulSoup(page, 'html.parser')
    news = soup.find_all('div', {'class': 'info-preview'})

    # создаем список blocks, где будут храниться все собранные данные

    blocks = []

    for n in news:
        try:
            blocks.append(parse_news_page_block(n))
        except Exception as e:
                pass

    return blocks

In [ ]:
def parse_news_page_block(one_block):

    # со страницы с рецептами собираются данные
    # для каждого поста и добавляются в словарь block

    block = {}

    title_obj = one_block.find('a', class_='h5')
    block['название статьи'] = title_obj.text
    link = one_block.find('a', class_='h5')['href']
    block['ссылка на статью'] = f"https://1000.menu{link}"

    return parse_one_article(block)


In [ ]:
def run_all(n):

    blocks = []
    for i in tqdm(range(n)):
        blocks.extend(get_nth_page(i))

    return blocks

In [ ]:
blocks = run_all(35)

  0%|          | 0/35 [00:00<?, ?it/s]

In [ ]:
new = pd.DataFrame(blocks)
new = new.dropna(subset=['ингредиенты'])

In [ ]:
new = new[new['ингредиенты'].apply(lambda x: len(x) > 0)]

In [ ]:
user_agent = UserAgent()

# некоторые рецепты были записаны иначе, чтобы их не потерять, пришлось пройти еще раз
# по датафрейму и заполнить ячейки с текстами рецептов

for index, row in tqdm(new.iterrows(), total=len(new)):
    if row['рецепт'] == '' or pd.isnull(row['рецепт']):
        response = requests.get(row['ссылка на статью'], headers={'User-Agent': user_agent.random})
        soup = BeautifulSoup(response.text, 'html.parser')

        try:
            post = soup.find('div', {'class': 'instructions'}).text
            new.at[index, 'рецепт'] = post
        except Exception:
            pass



  0%|          | 0/5471 [00:00<?, ?it/s]

In [ ]:
part_1 = pd.read_csv("my_recipes.csv")

In [ ]:
# соединяем полученный датафрейм с данными из первого краулера
data = pd.concat([part_1, new])

In [ ]:
data.to_csv('/content/my_recipes_data.csv')